# Introduction to Bayesian Optimization
**Drew Gjerstad**

## Contents
 * [Introduction](#introduction)
 * [Motivation](#motivation)
     - [Theoretical Motivation](#theoretical-motivation)
     - [Applications](#applications)
 * [Optimization Foundations](#optimization-foundations)
 * [Bayesian Foundations](#bayesian-foundations)
 * [The Bayesian Approach](#the-bayesian-approach)
 * [Bayesian Optimization Workflow](#bayesian-optimization-workflow)
 * [References](#references)

## Introduction

## Motivation

### Theoretical Motivation

### Applications

## Optimization Foundations

## Bayesian Foundations

## The Bayesian Approach

## Bayesian Optimization Workflow

## References